**Example using Dynamic Time Warping for clustering. **

In [1]:
from itertools import combinations
import math
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, lil_matrix

In [2]:
def l2(xi, yi):
    return (xi - yi)**2

def dtw_fn(x, y):
    """ Compute dynamic time warping distances for two time series x and y."""
    r, c = len(x), len(y)
    max_window = 10 # max search window (smaller->more efficient but less exact)
    w = max(max_window, abs(r - c))  
    D = np.ones((r + 1, c + 1)) * np.inf
    D[0, 0] = 0.

    for i in range(r):
        for j in range(max(0, i - w), min(c, i + w)):
            D[i+1, j+1] = l2(x[i], y[j])

    for i in range(r):  # why loop twice?!!
        for j in range(max(0, i - w), min(c, i + w)):
            D[i+1, j+1] += min(D[i, j], D[i, j+1], D[i+1, j])

    D = D[1:, 1:]
    return math.sqrt(D[-1, -1])

In [3]:
# Example.
dtw_fn([0,2,1,1.5], [0,0,2,1,1,1])

0.5

In [4]:
def dtw(X):
    """ Compute dtw for all pairs of rows in a matrix."""
    distances = np.zeros((len(X), len(X)))
    for i, j in combinations(range(len(X)), 2):
        distances[i,j] = dtw_fn(X[i], X[j])
    return distances

In [5]:
# Sample data.
X = np.array([[0,1,2,3], [1,2,3,0,2], [1,2,3],
              [5,0,5,0,5], [5,0,5,0,5,5], [5,0,5,0,4],
              [6,6,6], [6,6,6,7], [1, 0, 6, 6]])

In [6]:
# Pad each row with leading zeros to make all have same length.
def pad_data(X):
    max_len = max(len(i) for i in X)
    Xnew = np.zeros((len(X), max_len))
    for i, row in enumerate(X):
        diff = max_len - len(row)
        if diff > 0:
            Xnew[i] = np.concatenate(([0] * (diff), row))
        else:
            Xnew[i] = row
    return Xnew
X = pad_data(X)
X

array([[ 0.,  0.,  0.,  1.,  2.,  3.],
       [ 0.,  1.,  2.,  3.,  0.,  2.],
       [ 0.,  0.,  0.,  1.,  2.,  3.],
       [ 0.,  5.,  0.,  5.,  0.,  5.],
       [ 5.,  0.,  5.,  0.,  5.,  5.],
       [ 0.,  5.,  0.,  5.,  0.,  4.],
       [ 0.,  0.,  0.,  6.,  6.,  6.],
       [ 0.,  0.,  6.,  6.,  6.,  7.],
       [ 0.,  0.,  1.,  0.,  6.,  6.]])

In [7]:
# Cluster into 3 clusters using DTW.
from sklearn.cluster import AgglomerativeClustering
clusterer = AgglomerativeClustering(n_clusters=3, affinity=dtw, linkage='average')
clusters = clusterer.fit_predict(X)
clusters

array([2, 2, 2, 0, 0, 0, 1, 1, 1])

In [8]:
def find_centroids(X, clusters):
    """Find the centroid of each cluster, defined as the cluster element that
    has the lowest distance to all other elements in the cluster. """
    clusterids = set(clusters)
    cluster2centroid = dict()
    for clusterid in clusterids:
        cluster = np.where(clusters == clusterid)[0]
        distances = dtw(X[cluster])
        centroid = cluster[np.argmin(row.sum() for row in distances)]
        cluster2centroid[clusterid] = centroid
    return cluster2centroid
cluster2centroid = find_centroids(X, clusters)
cluster2centroid

{0: 3, 1: 6, 2: 0}